In [38]:
import pandas as pd
import os
from datetime import datetime, timedelta
import shutil
import time
import pyperclip
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from selenium.common.exceptions import TimeoutException, NoSuchElementException

import warnings

import requests
from bs4 import BeautifulSoup


In [5]:
# 현재 작업 디렉토리를 가져오기
current_directory = os.getcwd()


# crawler for cafe
relative_path_cafe = r'..\240131_cafe.xlsx'
cafe_path = os.path.join(current_directory, relative_path_cafe)

In [6]:
# 'ContentsList.xlsx' 파일 불러오기
df = pd.read_excel(cafe_path)

In [26]:
article_ids=df['index']
article_ids

0      1102942
1      1102917
2      1102889
3      1103106
4      1103045
        ...   
217    1116910
218    1116897
219    1116878
220    1116876
221    1116831
Name: index, Length: 222, dtype: int64

In [36]:
## open Chrome -> naver -> login
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 브라우저 생성
browser = webdriver.Chrome(options=chrome_options)
browser.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')

try:
    WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#id')))
    time.sleep(3)
    nid = 'qli_2018'  # Your ID
    npw = '12341234!@'  # Your Password
    pyperclip.copy(nid)
    browser.find_element(By.CSS_SELECTOR, '#id').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    pyperclip.copy(npw)
    browser.find_element(By.CSS_SELECTOR, '#pw').send_keys(Keys.CONTROL + 'v')
    pyperclip.copy('')  # Clear clipboard for security
    browser.find_element(By.XPATH, '//*[@id="log.login"]').click()
    browser.get('https://cafe.naver.com/kimyoooo')  # Navigate to the target page

    # Add your code here to parse the page with BeautifulSoup or perform other actions

except Exception as e:
    print(f"Exception: {str(e)}")

In [39]:
article_data = {}  # 빈 딕셔너리를 초기화합니다.

for article_id in article_ids:
    url = f"https://cafe.naver.com/kimyoooo/{article_id}"
    browser.get(url)
    time.sleep(1)
    # Switch to iframe if applicable
    browser.switch_to.frame('cafe_main')

    try:
        # Wait for the element to be present on the page
        wait = WebDriverWait(browser, 10)
        content_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer > div > div.content.CafeViewer > div > div")))
    except (TimeoutException, NoSuchElementException):
        # If content_element is not found within the specified timeout or if any other exception occurs, continue to the next article_id
        continue

    # Get the inner HTML content of the element
    inner_html = content_element.get_attribute("innerHTML")

    # Use BeautifulSoup to parse the inner HTML
    soup = BeautifulSoup(inner_html, 'html.parser')

    # Extract text from the parsed HTML
    text = soup.get_text(strip=True)

    # Store the text in the dictionary with the article_id as the key
    article_data[article_id] = text

# Print or use the collected data as needed
print(article_data)



UnexpectedAlertPresentException: Alert Text: 삭제되었거나 없는 게시글입니다.
Message: unexpected alert open: {Alert text : 삭제되었거나 없는 게시글입니다.}
  (Session info: chrome=121.0.6167.86)
Stacktrace:
	GetHandleVerifier [0x00007FF66FEF5E42+3538674]
	(No symbol) [0x00007FF66FB14C02]
	(No symbol) [0x00007FF66F9C5AEB]
	(No symbol) [0x00007FF66FA4C55E]
	(No symbol) [0x00007FF66FA2EE53]
	(No symbol) [0x00007FF66F9FF514]
	(No symbol) [0x00007FF66FA00631]
	GetHandleVerifier [0x00007FF66FF26CAD+3738973]
	GetHandleVerifier [0x00007FF66FF7C506+4089270]
	GetHandleVerifier [0x00007FF66FF74823+4057299]
	GetHandleVerifier [0x00007FF66FC45C49+720121]
	(No symbol) [0x00007FF66FB2126F]
	(No symbol) [0x00007FF66FB1C304]
	(No symbol) [0x00007FF66FB1C432]
	(No symbol) [0x00007FF66FB0BD04]
	BaseThreadInitThunk [0x00007FFF13B5257D+29]
	RtlUserThreadStart [0x00007FFF150AAA58+40]


In [42]:
article_data


{1102942: '5도2촌의 꿈을 드디어 이루어 냈습니다~\u200b설레는 마음으로 ~~~~짐이 너무많은대 다 안들어가서 ㅠㅠ 필요한것만 꾸겨넣고 ㅋㅋ보리랑 굴비랑 와이프도 구겨넣고 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 출발~~~~~\u200b집에서 1시간20분 거리 세컨하우스로!!!!\u200b모든게 다 새거라 비닐 때는대도 너무 힘들었습니다 ㅠㅠ\u200b꽃도 받고~ 고구마도 받고 ㅎㅎ 이웃과 사장님이 너무너무 좋았습니다~\u200b비닐하우스 배수로 작업하고 어두워지기전에 밥먹을준비를~~\u200b보리굴비도 따뜻하게 옷입히고~~친구가 축하한다고 회 사와서~~~ 1차로 회먹고 육회먹고~~ ㅎㅎㅎ\u200b2차로 소고기 구워먹고~~~~불멍하면서 맥주한잔하고~~ 마쉬멜로 구워먹었습니다~ ㅎㅎ\u200b다음날 일어나서 불부터 피웁니다 ㅋㅋㅋ 고구마 구워서 보리굴비 줄려고~~\u200b해장 매운탕 진짜 끝내줍니다..\u200b더먹고싶어하는 보리\u200b밭에 물좀주고 잡초도뽑고~~쉬다가\u200b수육을 해먹습니다 ㅎㅎㅎ\u200b하우스정리하고~~~\u200b시간이 정말 빨리가네요~~~~~~~~~~~ ㅎㅎ\u200b첫 주말농장생활 너무너무 재밌었습니다~~~ 일기 끝~~~',
 1102917: '인터넷이라는 사이버 공간을 통해 얼굴도 모르는 사람에게 매물을 소개하는 입장을 감안해서예의를 갖춰 정성껏 아래 양식에 맞춰 자기소개를 해주세요.\u200b1. 자신이 구입하고자 하는 토지의 목적 및 규모(평수)  - 전원주택지. 농지. 임야 등--->서까래 살아있는 구옥 25평대토지 100~200평 사이 (좀 더 작더라도 구조만 잘 빠져있으면 80평대도 생각하고 있습니다.)\u200b2. 토지 매입에 가능한 매입금액.---> 5천만원 전 후\u200b3. 원하는 지역(1순위 2순위 3순위)---> 충남권 생각하고 있습니다!현재 직장때문에 천안에 거주중, 대전이 본가라 천안이나 대전 근방을 우선순위로 두고 있습니다.천안,대전 에서 40분 근방이면 좋을 것 같습니다.1순위. 아산2순위. 

In [43]:
article_list = [{'article_id': key, 'content': value} for key, value in article_data.items()]

df_article = pd.DataFrame(article_list)

df_article['content'] = df_article['content'].str.replace('\u200b', '').str.strip()

df_article.dropna(subset=['content'], inplace=True)

In [44]:
df_article

,article_id,content
0,1102942,5도2촌의 꿈을 드디어 이루어 냈습니다~설레는 마음으로 ~~~~짐이 너무많은대 다 ...
1,1102917,인터넷이라는 사이버 공간을 통해 얼굴도 모르는 사람에게 매물을 소개하는 입장을 감안...
2,1102889,자주못가니 겨울준비 철저하게 합니다...저희집은...온도 감지 조절용 라디에이터에 ...
3,1103106,올해 첨 겨울을 나는데요.갑자기 궁금해져서 문의드려요.다른분들은 몇평에 얼마 정도 ...
4,1103361,지금 집도 작게 짓지만더 작게 짖고 싶어요 미니멀하게요정도는셀프로 하기에 적당할거 ...
...,...,...
161,1116776,시골 세컨하우스에서 약200m떨어진 위치에 지목이대지인땅에 콘테이너 2개를설치하여창...
162,1116628,저희 (주)타이거우드는건축자재를 납품하는 자재업체입니다.본 카페에 업체회원으로서 매...
163,1116910,세컨하우스를 짓기 위해 땅을 알아보고 있는데요.밭 한가운데 땅을 분할하여 거래가 가...
164,1116897,가입인사드려요~세컨하우스구입하고자 많이 배우고 싶어 가입했습니다!!!정말 어마어마한...


In [46]:
df

,index,title,writer,date,view,comment
0,1102942,"< 2023년 11월 10일,11일,12일 > 설레는 주말농장의 시작 사진",서산 일개미,2023.11.28.,571,25.0
1,1102917,서까래 살아있는 구옥 매물 소개 부탁드립니다.,호호오고고유옹,2023.11.28.,597,2.0
2,1102889,NaN,땅스부대,2023.11.28.,1072,NaN
3,1103106,LPG가스 보일러 한달에 얼마 정도 나오시나요?,당근구이,2023.11.29.,812,13.0
4,1103045,겨울에도 이용할 수 있는 소형별장 분양 사진링크,아인스홈,2023.11.29.,2583,NaN
...,...,...,...,...,...,...
217,1116910,세컨하우스 부지로 밭 한가운데 땅 어떤가요?,여전하군,2024.01.31.,357,10.0
218,1116897,안녕하세요~~,은똥79,2024.01.31.,22,5.0
219,1116878,창고 사이즈 고민입니다 ㅠㅠ 도와주세요 ! 사진,5도2촌고고,2024.01.31.,281,19.0
220,1116876,이동식 주택으로 추억을 만들고 싶네요.,탱자사랑,2024.01.31.,360,12.0


In [47]:
df_merged = pd.merge(df, df_article, left_on='index', right_on='article_id', how = 'left')

In [52]:
df_merged_without_id = df_merged.drop('article_id', axis=1)
df_merged_without_id.to_excel(cafe_path)